 7.1  Create a crawler using the webcrawler provided and discussed in my jupyter notebook ( https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb) , have it crawl the first 200 pages from a base URL of your choosing
 
7.2  Create a Stochastic matrix from its resulting crawling as per https://cs7083.wordpress.com/2013/01/31/demystifying-the-pagerank-and-hits-algorithms/

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

7.x For extra credit use the hits algorithm ( with a connectivity matrix )

7.xx For advanced credit find another ranking alogrithm and demonstrate that as wel

In [3]:
cd SB_LinkAnalysis_Project


C:\Users\AsusGo\Desktop\big-data\Lectures\SB_LinkAnalysis_Project


In [4]:
# %load SB_LinkAnalysis_Project/SB_LinkAnalysis_Project/items.py
# Define here the models for your scraped items
#
# See documentation in:
# http://doc.scrapy.org/en/latest/topics/items.html

import scrapy


class SBLinkAnalysisItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    pass

## Adding container to hold scraped data using scrapy.Item as the parent class
class CourseeraItem(scrapy.Item):
    title = scrapy.Field()
    link = scrapy.Field()
    desc = scrapy.Field()

In [5]:
# %load SB_LinkAnalysis_Project/SB_LinkAnalysis_Project/spiders/coursera_spider.py
import scrapy

class CeSpider(scrapy.Spider):
    name = "coursera"
    allowed_domains = [".coursera.org"]
    start_urls = ["https://www.coursera.org/browse/computer-science?languages=en","https://www.coursera.org/browse/data-science?languages=en"]

    def parse(self, response):
                  f.write(response.body)


In [12]:
cd SB_LinkAnalysis_Project

C:\Users\AsusGo\Desktop\big-data\Lectures\SB_LinkAnalysis_Project\SB_LinkAnalysis_Project


In [20]:
import requests
from scrapy.http import TextResponse

#running scrappy to get a response from a URL
r = requests.get('https://www.coursera.org/','https://www.coursera.org/browse/computer-science?languages=en')
response = TextResponse(r.url, body=r.text, encoding='utf-8')

In [21]:
print response

<200 https://www.coursera.org/?https://www.coursera.org/browse/computer-science?languages=en>


In [22]:
#Using xpath to access data
response.xpath('//title')

[<Selector xpath='//title' data=u'<title data-react-helmet="true">Coursera'>]

In [23]:
response.xpath('//title/text()').extract()

[u'Coursera | Online Courses From Top Universities. Join for Free']

In [24]:
response.xpath('//ul/li')

[<Selector xpath='//ul/li' data=u'<li class="c-ph-right-nav-button c-ph-ri'>,
 <Selector xpath='//ul/li' data=u'<li class="c-ph-right-nav-button c-ph-ri'>,
 <Selector xpath='//ul/li' data=u'<li class="c-ph-right-nav-button c-ph-en'>,
 <Selector xpath='//ul/li' data=u'<li class="c-ph-right-nav-button c-ph-lo'>,
 <Selector xpath='//ul/li' data=u'<li class="c-ph-right-nav-button c-ph-si'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1521"><div class="body'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1523"><a class="dim" h'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1525"><a class="dim" h'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1527"><a class="dim" h'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1529"><a class="dim" h'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1531"><a class="dim" h'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1533"><a class="dim" h'>,
 <Selector xpath='//ul/li' data=u'<li data-reactid="1535"><a cla

In [26]:
response.xpath('//ul/li/a/@href').extract()

[u'/browse',
 u'/courses',
 u'/enterprise',
 u'/?authMode=login&https%3A%2F%2Fwww.coursera.org%2Fbrowse%2Fcomputer-science%3Flanguages=en',
 u'/?authMode=signup&https%3A%2F%2Fwww.coursera.org%2Fbrowse%2Fcomputer-science%3Flanguages=en',
 u'https://about.coursera.org/',
 u'https://about.coursera.org/leadership',
 u'/about/careers',
 u'/browse',
 u'/professional-certificate',
 u'/degrees',
 u'/enterprise',
 u'/government',
 u'/about/partners',
 u'http://www.coursera.community/#mentor',
 u'http://www.coursera.community/#gtc',
 u'https://building.coursera.org/developer-program/',
 u'http://www.coursera.community/#beta-tester',
 u'http://blog.coursera.org',
 u'https://www.facebook.com/Coursera',
 u'https://www.linkedin.com/company/coursera',
 u'http://twitter.com/coursera',
 u'https://plus.google.com/+Coursera',
 u'https://building.coursera.org',
 u'/about/terms',
 u'/about/privacy',
 u'https://learner.coursera.help/hc',
 u'https://learner.coursera.help/hc/en-us/articles/208280056-Accommoda

In [27]:
# %load -r 1-20  code/Scrapy-Samples-master/crawlspider/items.csv

In [29]:
scrapy runspider coursera_spider.py

SyntaxError: invalid syntax (<ipython-input-29-35f3f0b14138>, line 1)

References:

 https://github.com/pschragger/big-data-python-class/blob/master/Lectures/Lecture_7_-_Link_Analysis.ipynb
 
 https://www.youtube.com/watch?v=O_j3OTXw2_E
 
 Source:
 
 https://www.coursera.org/
 